# Scraping the SEC Litigations Page

Importing helpful tools

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import requests

from bs4 import BeautifulSoup
import xml.etree.cElementTree as et

from IPython.display import display, Latex, Markdown


# RSS Feed

#### Let's first start with scraping the RSS feed which is a live, neatly formatted update of litigations of the SEC. Here is the link I'll be using: https://www.sec.gov/rss/litigation/litreleases.xml


In [9]:
parsedXML = et.parse("litigationData/sec11.xml")

In [10]:
for node in parsedXML.getroot():
    item = node.attrib.get('item')
    title = node.find('title')
    link = node.find('link')
    description = node.find('description')
    guid = node.find('guid')
    pubDate = node.find('pubDate')

In [11]:
def getvalueofnode(node):
    """ return node text or None """
    return node.text if node is not None else None

In [12]:
##Had to delete the <CHANNEL> tag from the file but at least it works 

parsed_xml = et.parse("litigationRSS/RSS_oct_4.xml")
dfcols = ['title', 'link', 'description', 'guid', 'pubDate']
df_xml = pd.DataFrame(columns=dfcols)
 
for node in parsed_xml.getroot():
    item = node.attrib.get('item')
    title = node.find('title')
    link = node.find('link')
    description = node.find('description')
    guid = node.find('guid')
    pubDate = node.find('pubDate')

    df_xml = df_xml.append(
        pd.Series([getvalueofnode(title), getvalueofnode(link),
                   getvalueofnode(description), getvalueofnode(guid),
                   getvalueofnode(pubDate)], index=dfcols), ignore_index=True)

In [24]:
df_xml.head(5)

,title,link,description,guid,pubDate
0,Christopher J. Spencer and John Busshaus,https://www.sec.gov/litigation/litreleases/201...,SEC Charges CEO and CFO of Digital Entertainme...,LR-24636,"Fri, 04 Oct 2019 11:20:02 EDT"
1,"PlexCorps, Dominic Lacroix, and Sabrina Paradi...",https://www.sec.gov/litigation/litreleases/201...,Defendants Charged in Fraudulent ICO to Pay Ne...,LR-24635,"Wed, 02 Oct 2019 16:54:27 EDT"
2,"Woojae (""Steve"") Jung, et al.",https://www.sec.gov/litigation/litreleases/201...,SEC Obtains Final Judgment Against Investment ...,LR-24634,"Wed, 02 Oct 2019 08:54:06 EDT"
3,"Westport Capital Markets, LLC and Christopher ...",https://www.sec.gov/litigation/litreleases/201...,SEC Obtains Partial Summary Judgment Against I...,LR-24633,"Wed, 02 Oct 2019 08:18:02 EDT"
4,"Bluepoint Investment Counsel, et al.",https://www.sec.gov/litigation/litreleases/201...,SEC Announces Fraud Charges Related to Wiscons...,LR-24632,"Mon, 30 Sep 2019 17:17:42 EDT"


### Discovered that the RSS feed is just a subscription feed and does not include everything - still helpful to know and have the code to parse it in case. Now let's try scraping the HTML table on the webpage.

In [25]:
import urllib
import urllib.request
from bs4 import BeautifulSoup 

## Attempt 1:  Sucesfully scraped; but not ideal format

In [26]:
def make_soup(url):
    the_page = requests.get(url)
    soup_data = BeautifulSoup(the_page.content, "html.parser")
    return soup_data

### Starting with 2010:

In [27]:
soup2010 = make_soup('https://www.sec.gov/litigation/litreleases/litrelarchive/litarchive2010.shtml')

In [28]:
data = []
table = soup2010.find_all('table')[4] # target the specific table
header, *rows = table.find_all('tr')

for row in rows:
    try:
        litigation, date, complaint = row.find_all('td')
    except ValueError:
        continue # ignore header row and quarter rows

    id = litigation.text.strip().split('-')[-1]
    date = date.text.strip()
    desc = complaint.text.strip().split('\t')[0]
    lit_url = litigation.find('a').get('href')

    try:
        comp_url = complaint.find('a').get('href')
    except AttributeError:
        comp_ulr = None # complaint url is optional

    info = dict(id=id, date=date, desc=desc, lit_url=lit_url, comp_url=comp_url)
    data.append(info)

In [30]:
data[0:5]

[{'id': '21795',
  'date': 'Dec 27, 2010',
  'desc': 'Alcatel-Lucent, S.A.',
  'lit_url': '/litigation/litreleases/2010/lr21795.htm',
  'comp_url': '/litigation/complaints/2010/comp21795.pdf'},
 {'id': '21794',
  'date': 'Dec 23, 2010',
  'desc': 'One or More Unknown Purchasers of Options of InterMune, Inc.',
  'lit_url': '/litigation/litreleases/2010/lr21794.htm',
  'comp_url': '/litigation/complaints/2010/comp21794.pdf'},
 {'id': '21793',
  'date': 'Dec 23, 2010',
  'desc': 'Michael E. Kelly, et al.',
  'lit_url': '/litigation/litreleases/2010/lr21793.htm',
  'comp_url': '/litigation/complaints/2010/comp21794.pdf'},
 {'id': '21792',
  'date': 'Dec 23, 2010',
  'desc': 'One or More Unknown Purchasers of Securities of Martek Biosciences Corporation',
  'lit_url': '/litigation/litreleases/2010/lr21792.htm',
  'comp_url': '/litigation/complaints/2010/comp21794.pdf'},
 {'id': '21791',
  'date': 'Dec 22, 2010',
  'desc': 'Pharma Holdings, Inc., Edward Klapp IV and Edward Klapp Jr.',
  'lit

# Attempt 2: Placing into a pandas dataframe; mostly functional

### SEC 2010

In [37]:
from bs4 import BeautifulSoup 
import requests, re

def make_soup(url):
    the_page = requests.get(url)
    soup_data = BeautifulSoup(the_page.content, "html.parser")
    return soup_data

soup2010 = make_soup('https://www.sec.gov/litigation/litreleases/litrelarchive/litarchive2010.shtml')
releases = []
links = []
dates = []
descs = [] 
addit_urls = []

for i in soup2010.select('td:nth-of-type(1):has([href^="/litigation/litreleases/"])'):
    sib_sib = i.next_sibling.next_sibling.next_sibling.next_sibling
    releases+= [i.a.text]
    links+= [i.a['href']]
    dates += [i.next_sibling.next_sibling.text.strip()]
    descs += [re.sub('\t+|\s+',' ',sib_sib.text.strip())]
    addit_urls += ['N/A' if sib_sib.a is None else sib_sib.a['href']]

result = list(zip(releases, links, dates, descs, addit_urls))


In [38]:
fulllink = []
for i in links:
    fulllink += ['https://www.sec.gov'+i]
    
additional_url = []
for i in addit_urls:
    if i == 'N/A':
        additional_url += ['N/A']
    else: 
        additional_url += ['https://www.sec.gov' + i]

In [39]:
sec2010 = pd.DataFrame({'title' : releases,
                   'links' : fulllink,
                   'dates' : dates,
                   'descs' : descs,
                   'additional url' : additional_url})

In [40]:
sec2010.head(5)

,title,links,dates,descs,additional url
0,LR-21795,https://www.sec.gov/litigation/litreleases/201...,"Dec 27, 2010","Alcatel-Lucent, S.A. See also: SEC Complaint",https://www.sec.gov/litigation/complaints/2010...
1,LR-21794,https://www.sec.gov/litigation/litreleases/201...,"Dec 23, 2010",One or More Unknown Purchasers of Options of I...,https://www.sec.gov/litigation/complaints/2010...
2,LR-21793,https://www.sec.gov/litigation/litreleases/201...,"Dec 23, 2010","Michael E. Kelly, et al.",N/A
3,LR-21792,https://www.sec.gov/litigation/litreleases/201...,"Dec 23, 2010",One or More Unknown Purchasers of Securities o...,N/A
4,LR-21791,https://www.sec.gov/litigation/litreleases/201...,"Dec 22, 2010","Pharma Holdings, Inc., Edward Klapp IV and Edw...",https://www.sec.gov/litigation/complaints/2010...


## 2014

In [41]:
soup2014 = make_soup('https://www.sec.gov/litigation/litreleases/litrelarchive/litarchive2014.shtml')

releases = []
links = []
dates = []
descs = [] 
addit_urls = []

for i in soup2014.select('td:nth-of-type(1):has([href^="/litigation/litreleases/"])'):
    sib_sib = i.next_sibling.next_sibling.next_sibling.next_sibling
    releases+= [i.a.text]
    links+= [i.a['href']]
    dates += [i.next_sibling.next_sibling.text.strip()]
    descs += [re.sub('\t+|\s+',' ',sib_sib.text.strip())]
    addit_urls += ['N/A' if sib_sib.a is None else sib_sib.a['href']]

result = list(zip(releases, links, dates, descs, addit_urls))

fulllink = []
for i in links:
    fulllink += ['https://www.sec.gov'+i]
    
additional_url = []
for i in addit_urls:
    if i == 'N/A':
        additional_url += ['N/A']
    else: 
        additional_url += ['https://www.sec.gov' + i]

In [42]:
fulllink = []
for i in links:
    fulllink += ['https://www.sec.gov'+i]
    
additional_url = []
for i in addit_urls:
    if i == 'N/A':
        additional_url += ['N/A']
    else: 
        additional_url += ['https://www.sec.gov' + i]

In [43]:
sec2014 = pd.DataFrame({'title' : releases,
                   'links' : fulllink,
                   'dates' : dates,
                   'descs' : descs,
                   'additional url' : additional_url})

In [44]:
sec2014.head(5)

,title,links,dates,descs,additional url
0,LR-23169,https://www.sec.gov/litigation/litreleases/201...,"Dec. 23, 2014",Efstratios “Elias” D. Argyropoulos and Prima C...,N/A
1,LR-23168,https://www.sec.gov/litigation/litreleases/201...,"Dec. 23, 2014","E-Monee.com Inc., et al.",N/A
2,LR-23167,https://www.sec.gov/litigation/litreleases/201...,"Dec. 22, 2014",Shivbir S. Grewal and Preetinder Grewal,N/A
3,LR-23166,https://www.sec.gov/litigation/litreleases/201...,"Dec. 22, 2014",Howard B. Present See also: SEC Complaint,https://www.sec.gov/litigation/complaints/2014...
4,LR-23165,https://www.sec.gov/litigation/litreleases/201...,"Dec. 19, 2014","Frank A. Dunn, et al. Other Release No.:",N/A


## 2015

In [45]:
soup2015 = make_soup('https://www.sec.gov/litigation/litreleases/litrelarchive/litarchive2015.shtml')

releases = []
links = []
dates = []
descs = [] 
addit_urls = []

for i in soup2015.select('td:nth-of-type(1):has([href^="/litigation/litreleases/"])'):
    sib_sib = i.next_sibling.next_sibling.next_sibling.next_sibling
    releases+= [i.a.text]
    links+= [i.a['href']]
    dates += [i.next_sibling.next_sibling.text.strip()]
    descs += [re.sub('\t+|\s+',' ',sib_sib.text.strip())]
    addit_urls += ['N/A' if sib_sib.a is None else sib_sib.a['href']]

result = list(zip(releases, links, dates, descs, addit_urls))


In [46]:
fulllink = []
for i in links:
    fulllink += ['https://www.sec.gov'+i]
    
additional_url = []
for i in addit_urls:
    if i == 'N/A':
        additional_url += ['N/A']
    else: 
        additional_url += ['https://www.sec.gov' + i]

In [47]:
sec2015 = pd.DataFrame({'title' : releases,
                   'links' : fulllink,
                   'dates' : dates,
                   'descs' : descs,
                   'additional url' : additional_url})

In [48]:
sec2015.head(5)

,title,links,dates,descs,additional url
0,LR-23440,https://www.sec.gov/litigation/litreleases/201...,"Dec. 28, 2015","New Stream Capital, LLC, et al. Other Release ...",N/A
1,LR-23439,https://www.sec.gov/litigation/litreleases/201...,"Dec. 23, 2015",Michael J. Ling,N/A
2,LR-23438,https://www.sec.gov/litigation/litreleases/201...,"Dec. 23, 2015","Bonan Huang, et al.",N/A
3,LR-23437,https://www.sec.gov/litigation/litreleases/201...,"Dec. 23, 2015",In re Four Points Capital Partners LLC,N/A
4,LR-23436,https://www.sec.gov/litigation/litreleases/201...,"Dec. 22, 2015","Southern Cross Resources Group, Inc., et al. S...",https://www.sec.gov/litigation/complaints/2015...


# III. Converting from DF to Excel

In [35]:
from pandas import ExcelWriter


writer = ExcelWriter('sec2010litigations.xlsx')
sec2010.to_excel(writer)
writer.save()

# DF TO CSV: Only run when necessary to update
#sec2010.to_csv('sec2010litigations.xlsx', sep=',')

In [36]:
from pandas import ExcelWriter


writer2 = ExcelWriter('sec2014litigations.xlsx')
sec2014.to_excel(writer2)
writer2.save()

# DF TO CSV: Only run when necessary to update
#sec2014.to_csv('sec2014litigations.xlsx', sep=',')

NameError: name 'sec2014' is not defined

In [134]:
from pandas import ExcelWriter


writer3 = ExcelWriter('sec2015litigations.xlsx')
sec2015.to_excel(writer3)
writer3.save()

# DF TO CSV: Only run when necessary to update
sec2015.to_csv('sec2015litigations.xlsx', sep=',')